In [ ]:
import sqlite3
import json
from pathlib import Path
import pandas as pd
db_path = "../directus-quran/Quran_111.db"
conn = sqlite3.connect(db_path)

query = """
SELECT sura_no, sura_name_en, sura_name_roman, ayah_no_surah, ayah_translation_en
FROM TheNobleQuran
ORDER BY sura_no, ayah_no_surah
"""

df = pd.read_sql_query(query, conn)
conn.close()

# Grouping and formatting for FlexSearch
search_index = []
for surah_no, group in df.groupby("sura_no"):
    surah_name = group["sura_name_en"].iloc[0]
    surah_roman = group["sura_name_roman"].iloc[0]
    content = "\n\n".join(
        f"**{row['ayah_no_surah']}** {row['ayah_translation_en']}" 
        for _, row in group.iterrows()
    )
    search_index.append({
        "id": int(surah_no),
        "title": f"Surah {surah_no:03d} - {surah_name} ({surah_roman})",
        "slug": f"/surahs/surah-{surah_no:03d}",
        "content": content
    })

# Save to public directory
# output_path = Path("astro-quran/public/search/flexsearch-index.json")
output_path = Path("../astro-quran/public/search/flexsearch-index.json")
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(search_index, f, ensure_ascii=False, indent=2)

 

search_index[:2]  # Preview first 2 entries for verification